# Tiling csv-files for GridViz  
see file format specification at  
https://eurostat.github.io/gridviz/docs/tiledformat.html

In [1]:
import pandas as pd
import math
import os
pd.options.mode.chained_assignment = None

In [2]:
griddf = pd.read_csv("whgHeizTyp100m2011lowerleft00.csv")

In [3]:
griddf.head(3)

,x,y,Insg,fernw,etagen,zentral
0,4335700,2686500,3,NaN,NaN,3.0
1,4337000,2689100,3,NaN,NaN,NaN
2,4341200,2691200,3,NaN,NaN,3.0


In [4]:
tilingBounds= {
        "yMin": griddf["y"].min(),
        "yMax": griddf["y"].max(),
        "xMax": griddf["x"].max(),
        "xMin": griddf["x"].min()
    }

In [5]:
resolutionGeo=100

In [6]:
tileSizeCell=250

In [7]:
xIncrements = math.ceil((tilingBounds["xMax"]-tilingBounds["xMin"])/(tileSizeCell*resolutionGeo))
yIncrements = math.ceil((tilingBounds["yMax"]-tilingBounds["yMin"])/(tileSizeCell*resolutionGeo))

In [8]:
xIncrements, yIncrements

(26, 35)

In [19]:
# create numbered folders for xIncrements
for i in range(xIncrements):
    
    newpath = r'./tiled_data/'+str(i) 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    # filter data on x-axis
    tmpXdf = griddf[(griddf.x>=tilingBounds["xMin"]+(i*tileSizeCell*resolutionGeo)) & 
                (griddf.x<(tilingBounds["xMin"]+(i+1)*tileSizeCell*resolutionGeo))]
        
    # iterate over the y-axis
    for j in range(yIncrements):
        
        # filter data to tile
        tmpXYdf = tmpXdf[(tmpXdf.y>=tilingBounds["yMin"]+(j*tileSizeCell*resolutionGeo)) & 
                    (tmpXdf.y<(tilingBounds["yMin"]+(j+1)*tileSizeCell*resolutionGeo))]

        # reduce grid coordinates to reference tile instead of originPoint
        tmpXYdf["x"] = tmpXYdf["x"].apply(lambda n: (n - tilingBounds["xMin"]
                                                     - i*tileSizeCell*resolutionGeo)/resolutionGeo)
        tmpXYdf["y"] = tmpXYdf["y"].apply(lambda n: (n - tilingBounds["yMin"]
                                                     - j*tileSizeCell*resolutionGeo)/resolutionGeo)

        # write to csv if tile contains cells
        if len(tmpXYdf)>0:
            
            tmpXYdf.to_csv(f'./tiled_data/{i}/{j}.csv', float_format='%.0f', index=False)